In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch



In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write)

In [ ]:

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="./cache")

llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="./cache",
    torch_dtype=torch.float16,                              # Use half precision if supported
    device_map="auto",                                    
    low_cpu_mem_usage=True                                  
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Role Prompting
- Roles give context to LLMs what type of answers are desired.


### Prompting without a role

In [4]:
prompt = """
How can I answer this question from my friend:
What is the meaning of life?
"""
inputs = tokenizer(prompt, return_tensors="pt")
outputs = llm.generate(**inputs, max_length=200)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



How can I answer this question from my friend:
What is the meaning of life?
--------------------------------------------------------

You can answer this question with a philosophical or scientific perspective, or a personal one.

### Philosophical Perspective

The meaning of life is a question that has puzzled philosophers and thinkers for centuries. Some possible answers include:

*   **Existentialism**: The meaning of life is to create one's own meaning. It is up to each individual to find their own purpose and create their own values.
*   **Nihilism**: The meaning of life is nothing. Life has no inherent meaning, and it is up to each individual to create their own meaning.
*   **Absurdism**: The meaning of life is to find ways to live with the absurdity of the human condition. We must find ways to make sense of a seemingly meaningless world.

### Scientific Perspective

From a scientific perspective, the meaning of life can be seen as the pursuit of survival and reproduction. The 

### Prompting with a role

In [5]:
role = """
Your role is a life coach \
who gives advice to people about living a good life.\
You attempt to provide unbiased advice.
You respond in the tone of an English pirate.
"""

prompt_with_role = f"""
{role}
How can I answer this question from my friend:
What is the meaning of life?
"""

inputs = tokenizer(prompt_with_role, return_tensors="pt")
outputs = llm.generate(**inputs, max_length=200)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




Your role is a life coach who gives advice to people about living a good life.You attempt to provide unbiased advice.
You respond in the tone of an English pirate.

How can I answer this question from my friend:
What is the meaning of life?
Aristotle said it was happiness, but I don't know if I agree with him.

Shiver me timbers! Yer lookin' fer a way to tackle that mighty question, eh? Alright then, matey! Let's set sail fer a treasure-filled discussion!

First, ye gotta understand that the meaning o' life be a mighty subjective question. What gives life meaning to one swashbuckler might be a mere trifle to another. Aristotle's take on happiness be a good starting point, but ye shouldn't be afraid to chart yer own course.

Here be a few things to consider, me hearty:

1. **What makes ye tick?** What gets ye excited, what makes ye feel alive? Is


### Summarization

In [6]:
text="""LLaMA was announced on February 24, 2023, via a blog post and a paper describing the model's training, architecture, and performance. The inference code used to run the model was publicly released under the open-source GPLv3 license.
Access to the model's weights was managed by an application process, with access to be granted "on a case-by-case basis to academic researchers; those affiliated with organizations in government, civil society, and academia; and industry research laboratories around the world".

Llama was trained on only publicly available information, and was trained at various model sizes, with the intention to make it more accessible to different hardware.

Meta AI reported the 13B parameter model performance on most NLP benchmarks exceeded that of the much larger GPT-3 (with 175B parameters), and the largest 65B model was competitive with state of the art models such as PaLM and Chinchilla.

On July 18, 2023, in partnership with Microsoft, Meta announced Llama 2, the next generation of Llama. Meta trained and released Llama 2 in three model sizes: 7, 13, and 70 billion parameters.
The model architecture remains largely unchanged from that of LLaMA-1 models, but 40% more data was used to train the foundational models. The accompanying preprint also mentions a model with 34B parameters that might be released in the future upon satisfying safety targets.
Llama 2 includes foundation models and models fine-tuned for chat. In a further departure from LLaMA, all models are released with weights and are free for many commercial use cases. However, due to some remaining restrictions, Meta's description of LLaMA as open source has been disputed by the Open Source Initiative (known for maintaining the Open Source Definition).
Code Llama is a fine-tune of Llama 2 with code specific datasets. 7B, 13B, and 34B versions were released on August 24, 2023, with the 70B releasing on the January 29, 2024. Starting with the foundation models from Llama 2, Meta AI would train an additional 500B tokens of code datasets, before an additional 20B token of long-context data, creating the Code Llama foundation models.
This foundation model was further trained on 5B instruction following token to create the instruct fine-tune.
Another foundation model was created for Python code, which trained on 100B tokens of Python-only code, before the long-context data.
On April 18, 2024, Meta released Llama-3 with two sizes: 8B and 70B parameters.
The models have been pre-trained on approximately 15 trillion tokens of text gathered from “publicly available sources” with the instruct models fine-tuned on “publicly available instruction datasets, as well as over 10M human-annotated examples".
Meta AI's testing showed in April 2024 that Llama 3 70B was beating Gemini pro 1.5 and Claude 3 Sonnet on most benchmarks. Meta also announced plans to make Llama 3 multilingual and multimodal, better at coding and reasoning, and to increase its context window.
During an interview with Dwarkesh Patel, Mark Zuckerberg said that the 8B version of Llama 3 was nearly as powerful as the largest Llama 2.
Compared to previous models, Zuckerberg stated the team was surprised that the 70B model was still learning even at the end of the 15T tokens training.
The decision was made to end training to focus GPU power elsewhere.
"""

In [7]:
prompt = f"""
Summarize this text and extract some key points.
What did the author say about llama models?:

text: {text}
"""
inputs = tokenizer(prompt, return_tensors="pt")
outputs = llm.generate(**inputs, max_length=800)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Summarize this text and extract some key points.
What did the author say about llama models?:

text: LLaMA was announced on February 24, 2023, via a blog post and a paper describing the model's training, architecture, and performance. The inference code used to run the model was publicly released under the open-source GPLv3 license.
Access to the model's weights was managed by an application process, with access to be granted "on a case-by-case basis to academic researchers; those affiliated with organizations in government, civil society, and academia; and industry research laboratories around the world".

Llama was trained on only publicly available information, and was trained at various model sizes, with the intention to make it more accessible to different hardware.

Meta AI reported the 13B parameter model performance on most NLP benchmarks exceeded that of the much larger GPT-3 (with 175B parameters), and the largest 65B model was competitive with state of the art models such a

##  Providing New Information in the Prompt

### I will ask the model about something that happened in October 2024, but since LLaMA 3.1 was released in July 2024, it wouldn’t have known about it.

In [8]:
prompt= f"""How might Bisan’s courage and determination impact others who hear her story,
 and what can we learn from her strength in the face of such hardship?
 """
inputs = tokenizer(prompt, return_tensors="pt")
outputs = llm.generate(**inputs, max_length=512)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


How might Bisan’s courage and determination impact others who hear her story,
 and what can we learn from her strength in the face of such hardship?
  1. Bisan’s story has the potential to inspire and empower others who hear it, particularly young people who may be facing similar challenges. Her courage and determination in the face of adversity can motivate others to stand up for their rights and fight against injustice.
  2. Bisan’s story highlights the importance of resilience and the human spirit’s ability to adapt and overcome even the most difficult circumstances. Her strength and determination can serve as a reminder that even in the darkest of times, there is always hope for a better future.
  3. Bisan’s story also underscores the need for support and solidarity in the face of oppression. Her experience of being a refugee and facing discrimination and marginalization highlights the importance of building a supportive community and advocating for the rights of marginalized group

In [9]:
context="""I want to tell you about a little girl I saw on TV. She was 8 years old, and her name was Bisan.
 Despite her young age, she was incredibly wise and strong.
 She was carrying her injured sister through the scorching heat under the hot sun. Having lost her family, she was desperately searching for a hospital to help her sister recover. As she prayed to God for help, a kind man approached her and asked where she was going.
 She told him, and he offered to help her get to the hospital. This happened just a couple of days ago
 """

In [10]:
prompt_with_info= f"""How might Bisan’s courage and determination impact others who hear her story,
 and what can we learn from her strength in the face of such hardship?
 context: {context}
 """
inputs = tokenizer(prompt_with_info, return_tensors="pt")
outputs = llm.generate(**inputs, max_length=512)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


How might Bisan’s courage and determination impact others who hear her story,
 and what can we learn from her strength in the face of such hardship?
 context: I want to tell you about a little girl I saw on TV. She was 8 years old, and her name was Bisan.
 Despite her young age, she was incredibly wise and strong.
 She was carrying her injured sister through the scorching heat under the hot sun. Having lost her family, she was desperately searching for a hospital to help her sister recover. As she prayed to God for help, a kind man approached her and asked where she was going.
 She told him, and he offered to help her get to the hospital. This happened just a couple of days ago
 
 1.  Bisan’s courage and determination might inspire others to find the strength to carry on, even in the face of unimaginable hardship. Her resilience and willingness to help her sister could motivate people to persevere through their own challenges.
 
 2.  Her story could also raise awareness about the strug

## Chain-of-thought Prompting

In [11]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?
"""
inputs = tokenizer(prompt, return_tensors="pt")
outputs = llm.generate(**inputs, max_length=512)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?
Yes or No?

Answer: Yes.

Explanation: We can put 5 people in each of the two cars.  We can put 2 people on each of the two motorcycles.  The 5 people left over can get in one of the cars.  Therefore, we can get all 15 people to the restaurant.  The answer is yes.  (This is a trick question.  The question asks if we can get all 15 people to the restaurant by car or motorcycle.  The question does not say we have to use the cars and motorcycles to seat all 15 people.  We can use them to seat 13 people, and the other 2 people can walk.)  (Note:  This is a problem from the "Harvard-MIT Math Tournament" and is one of the most famous problems in the tournament.)  (Note:  This problem is also a good example of a "lateral thinking" problem.  Lateral thinking is the ability to think of soluti

- Modify the prompt to ask the model to "think step by step" about the math problem.

In [12]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?

Think step by step.
"""
inputs = tokenizer(prompt, return_tensors="pt")
outputs = llm.generate(**inputs, max_length=512)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?

Think step by step.
First, let's see how many people we can transport in the cars.
There are 2 cars. Each car can seat 5 people. So 2 x 5 = 10 people can be transported by car.
We still have 5 people left. We have 2 motorcycles, each of which can seat 2 people. 2 x 2 = 4 people can be transported by motorcycle.
We still have 1 person left. We can transport this person by car. We have 2 cars, each of which can seat 5 people. One car is empty, and the other car has 1 seat left.
We can all get to the restaurant by car or motorcycle. We can transport 9 people in the cars, and the remaining 6 people can be transported by motorcycle.

In this solution, we used the fact that we have 2 cars, and each car can seat 5 people. We used the fact that we have 2 motorcycles, and each motorcycle can

- Provide the model with additional instructions.

In [13]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?

Think step by step.
Explain each intermediate step.
Only when you are done with all your steps,
provide the answer based on your intermediate steps.
"""
inputs = tokenizer(prompt, return_tensors="pt")
outputs = llm.generate(**inputs, max_length=512)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?

Think step by step.
Explain each intermediate step.
Only when you are done with all your steps,
provide the answer based on your intermediate steps.

## Step 1: Calculate the total number of people that can be transported by cars.
There are 2 cars, and each car can seat 5 people. So, the total number of people that can be transported by cars is 2 * 5 = 10.

## Step 2: Calculate the total number of people that can be transported by motorcycles.
There are 2 motorcycles, and each motorcycle can fit 2 people. So, the total number of people that can be transported by motorcycles is 2 * 2 = 4.

## Step 3: Determine the remaining number of people that need transportation.
There are 15 people in total, and 10 can be transported by cars, and 4 can be transported by motorcycles. So, the numbe

- The order of instructions matters!
- Asking the model to "answer first" and "explain later"

In [14]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?
Think step by step.
Provide the answer as a single yes/no answer first.
Then explain each intermediate step.
"""
inputs = tokenizer(prompt, return_tensors="pt")
outputs = llm.generate(**inputs, max_length=512)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?
Think step by step.
Provide the answer as a single yes/no answer first.
Then explain each intermediate step.
 
YES

Step 1: We have 15 people who want to go to the restaurant.
Step 2: We have 2 cars, each can seat 5 people. That means the 2 cars can seat 10 people in total.
Step 3: We have 2 motorcycles, each can seat 2 people. That means the 2 motorcycles can seat 4 people in total.
Step 4: The number of people who can go by car is 10, which is less than the total number of people (15). So, we can't seat all 15 people in the cars. But, the number of people who can go by motorcycle is 4, which is also less than the total number of people (15).
Step 5: So, we can seat 10 people in the cars, and 4 people in the motorcycles. That leaves us with 15 - 10 - 4 = 1 person who can't go by car